# Una aplicación dentro de la máquina virtual

## Obtener roop desde github

In [ ]:
# Clonar el repositorio
!git clone https://github.com/s0md3v/roop

## Ajustar el funcionamiento en para un entorno virtal de python

In [ ]:
# Entrar a la carpeta del repositorio clonado
%cd roop

In [ ]:
# Crear una carpeta para separar las imagenes a procesar
!mkdir recursos

In [ ]:
# Se crea y se activa un entorno para python
!pip install virtualenv
!pip install gradio
!virtualenv .carificador

In [ ]:
# Instalar las librerías para que funcione
!source .carificador/bin/activate; pip install -r requirements.txt
!source .carificador/bin/activate; pip install matplotlib-inline

## Creando un script de bash para las facilitar las llamadas

In [ ]:
# Crear un script de bash para facilitar el uso
!echo "#!/bin/bash"                                    >  carificar.sh
!echo 'CARA=$1'                                        >> carificar.sh
!echo 'OBJETIVO=$2'                                    >> carificar.sh
!echo 'RESULTADO=$3'                                   >> carificar.sh
!echo "source /content/roop/.carificador/bin/activate" >> carificar.sh
!echo "echo ------- ejecutando ---------------------------" >> carificar.sh
!echo 'echo python run.py --many-faces --execution-provider cuda --frame-processor face_swapper face_enhancer -s "${CARA}" -t "${OBJETIVO}" -o "${RESULTADO}" ' >> carificar.sh
!echo "echo ----------------------------------------------" >> carificar.sh
!echo 'python run.py --many-faces --execution-provider cuda --frame-processor face_swapper face_enhancer -s "${CARA}" -t "${OBJETIVO}" -o "${RESULTADO}" ' >> carificar.sh
!chmod 755 carificar.sh
!ls -la

## Bajar el modelo que permite hacer los cambios

In [ ]:
# Bajar el modelo
!mkdir models
!ls -al models
!wget -O models/inswapper_128.onnx https://huggingface.co/ezioruan/inswapper_128.onnx/resolve/main/inswapper_128.onnx
!ls -al models

### Subir algunas imágenes para las pruebas

In [ ]:
# Poner unas plantillas para usar
%cd recursos
from google.colab import files
uploaded = files.upload()
%cd ..
!ls -la recursos

## Importar las libreríás para preparar la aplicación web

In [ ]:
# Librerías para las aplicaciones web y las funciones de la aplicación web
from IPython.display import Image
from PIL import Image as Pimagen
import os
import locale
import gradio as gr
import cv2

In [ ]:
# Localización para gradio
locale.getpreferredencoding = lambda: "UTF-8"

### Pruebas del script de bash con las imágenes subidas

Notar que si suben imágenes diferentes, ***cambiar los nombres de las imágenes*** tomando en cuenta las extensiones y manteniendo la ruta sugerida dentro de la carpeta **recursos**

In [ ]:
# hacer un cambio con el script de bash
!rm -f ./recursos/cambio.jpg
!bash carificar.sh ./recursos/yo.jpg ./recursos/retrato-hombre-01.jpg ./recursos/cambio.jpg
Image("./recursos/cambio.jpg")

## Definir una funcióñ para uso en código normal de python con llamada al sistema operativo

In [ ]:
# Hacer una función para realizar cambios
def cambiar( micara, objetivo ) :
  os.system( "rm -f './recursos/cambio.jpg' ")
  os.system( f"bash carificar.sh '{micara}'  '{objetivo}' './recursos/cambio.jpg'")
  return Pimagen.open(r"./recursos/cambio.jpg")

### Probar código en python para uso de la función de python

In [ ]:
# Hacer un cambio con la función
imagen_generada = cambiar("./recursos/yo.jpg", "./recursos/arqueologo.jpg")
imagen_generada.save( "./prueba.jpg" )
Image("./prueba.jpg")

## Definición de una función para envolver las llamadas a conversión desde gradio

In [ ]:
# Función envolvedora para que funcione la interfaz de gradio
def carificadorGr( imagen_01, imagen_02 ) :
  os.system( "rm -f /tmp/cara.jpg"      )
  os.system( "rm -f /tmp/objetivo.jpg"  )
  cv2.imwrite("/tmp/cara.jpg"     , cv2.cvtColor(imagen_01, cv2.COLOR_BGR2RGB) )
  cv2.imwrite("/tmp/objetivo.jpg" , cv2.cvtColor(imagen_02, cv2.COLOR_BGR2RGB) )
  devolver = cambiar( "/tmp/cara.jpg", "/tmp/objetivo.jpg" )
  return devolver

## Diseño de la interfaz para gradio

In [ ]:
# diseño de la interfaz de gradio
caja_de_entrada_1  = gr.Image( label = "Tu foto"       )
caja_de_entrada_2  = gr.Image( label = "Una plantilla" )
caja_de_salida     = gr.Image( label = "Resultado" )
titulo_de_mi_app   = "Carificador enchuchumustnado"
descripcion_app    = "Herramienta para enchuchumustar caras en plantillas"
interfaz = gr.Interface(
    fn             = carificadorGr      ,
    inputs         = [
      caja_de_entrada_1 ,
      caja_de_entrada_2
    ] ,
    outputs        = [ caja_de_salida  ] ,
    title          = titulo_de_mi_app    ,
    description    = descripcion_app     ,
    flagging_mode  = "never"
)

## Lanzar y probar la aplicación web

In [ ]:
#lanzar aplicación
interfaz.launch( share = True )

## Finalizar la aplicación web

In [ ]:
# cerrar pliacación web
interfaz.close(  )